In [ ]:
import os
import glob
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from google.colab import drive

from pycocotools.coco import COCO

from skimage import io, img_as_ubyte, exposure
from skimage.color import label2rgb
from skimage.transform import resize, rotate, AffineTransform, warp

from keras.saving import register_keras_serializable

from tensorflow.keras import backend as K
from tensorflow.keras import models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Activation
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Add, Dense,ReLU, GlobalAveragePooling2D, Reshape,Concatenate,Layer
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Suppress specific warnings for low contrast images
warnings.filterwarnings("ignore", message=".*is a low contrast image")

# Define base directories (update these paths as necessary)
base_dir = '/content/drive/MyDrive/CV_9517/Dataset'
data_dir = os.path.join(base_dir)
output_folder = os.path.join(base_dir, 'preprocess_2000')

# Define output directories for each split (train, val, and test)
train_image_mask_dir = os.path.join(output_folder, 'training')
val_image_mask_dir = os.path.join(output_folder, 'validation')
test_image_mask_dir = os.path.join(output_folder, 'testing')

train_image_dir = os.path.join(train_image_mask_dir, 'images')
train_mask_dir = os.path.join(train_image_mask_dir, 'masks')

val_image_dir = os.path.join(val_image_mask_dir, 'images')
val_mask_dir = os.path.join(val_image_mask_dir, 'masks')

test_image_dir = os.path.join(test_image_mask_dir, 'images')
test_mask_dir = os.path.join(test_image_mask_dir, 'masks')

# # Ensuring that all necessary directories exist
for directory in [train_image_mask_dir, val_image_mask_dir, test_image_mask_dir]:
    os.makedirs(directory, exist_ok=True)

for directory in [train_image_dir, train_mask_dir, val_image_dir, val_mask_dir, test_image_dir, test_mask_dir]:
    os.makedirs(directory, exist_ok=True)

print("Setup is complete. Directories created as needed.")

Setup is complete. Directories created as needed.


In [ ]:
# Initialize COCO API for annotations
annFile = os.path.join(data_dir, 'annotations.json')
coco = COCO(annFile)

# Load metadata
metadata_path = os.path.join(data_dir, 'metadata.csv')
metadata = pd.read_csv(metadata_path)

# Select 2500 random image IDs
all_image_ids = metadata['id'].tolist()
random.seed(42)  # For reproducibility
selected_image_ids = random.sample(all_image_ids, 2000)
subset_metadata = metadata[metadata['id'].isin(selected_image_ids)]

# Verify selection
print(f"Total images selected for processing: {len(subset_metadata)}")
print(subset_metadata.head())

loading annotations into memory...
Done (t=7.74s)
creating index...
index created!
Total images selected for processing: 2000
    id  width  height                   file_name            timestamp  \
6    7   2000    1333  images/t001/OiaMESWPyh.JPG  2014-07-14 14:52:17   
9   10   2000    1124  images/t001/ZKnMahXQgJ.JPG  2010-07-02 14:08:14   
21  22   2000    1333  images/t001/zgOfklUqJC.JPG  2014-07-14 14:48:36   
23  24   2000    1124  images/t002/AyPUCFEHdl.JPG  2010-07-05 17:15:15   
27  28   2000    1124  images/t002/SuPxFCvAob.jpg  2010-07-05 00:00:00   

   identity        date  
6      t001  2014-07-14  
9      t001  2010-07-02  
21     t001  2014-07-14  
23     t002  2010-07-05  
27     t002  2010-07-05  


In [ ]:
def determine_split(row):
    year = int(row['date'].split('-')[0])
    if year <= 2018:
        return 'train'
    elif year == 2019:
        return 'val'
    else:
        return 'test'

# Apply split determination
subset_metadata['split'] = subset_metadata.apply(determine_split, axis=1)

# Create separate metadata for each split
train_metadata = subset_metadata[subset_metadata['split'] == 'train']
val_metadata = subset_metadata[subset_metadata['split'] == 'val']
test_metadata = subset_metadata[subset_metadata['split'] == 'test']

# Print the counts to verify
print(f"Training set size: {len(train_metadata)}")
print(f"Validation set size: {len(val_metadata)}")
print(f"Test set size: {len(test_metadata)}")

Training set size: 1223
Validation set size: 259
Test set size: 518


<ipython-input-25-0a7dfceaa4b8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_metadata['split'] = subset_metadata.apply(determine_split, axis=1)


In [ ]:
def preprocess_image(image, target_size=(224, 224)):
    resized_image = resize(image, target_size, anti_aliasing=True)
    scaled_image = np.clip(resized_image, 0, 1)
    return scaled_image

def preprocess_mask(mask, target_size=(224, 224)):
    resized_mask = resize(mask, target_size, order=0, preserve_range=True, anti_aliasing=False)
    return resized_mask.astype(np.uint8)

def apply_augmentations(image, mask):
    if random.random() > 0.5:
        image = np.fliplr(image)
        mask = np.fliplr(mask)
    if random.random() > 0.5:
        image = np.flipud(image)
        mask = np.flipud(mask)

    # Random rotation
    angle = random.uniform(-10, 10)
    image = rotate(image, angle, mode='edge')
    mask = rotate(mask, angle, order=0, preserve_range=True)

    # Random translation
    shift_x = random.uniform(-10, 10)
    shift_y = random.uniform(-10, 10)
    tform = AffineTransform(translation=(shift_x, shift_y))
    image = warp(image, tform, mode='wrap')
    mask = warp(mask, tform, order=0, preserve_range=True)

    # Random brightness adjustment
    factor = random.uniform(0.97, 1.03)
    image = np.clip(image * factor, 0, 1)
    image = exposure.adjust_gamma(image, gamma=random.uniform(0.97, 1.03))

    return image, mask.astype(np.uint8)

print("Preprocessing and augmentation functions are ready.")

Preprocessing and augmentation functions are ready.


In [ ]:
def process_and_save_dataset_multiclass(coco, split_metadata, image_output_dir, mask_output_dir=None, augment=False, save_masks=True, colormap='viridis'):
    # Define the label priorities
    label_priority = {3: 2, 2: 1, 1: 0}  # Head > Flippers > Body

    for idx, row in split_metadata.iterrows():
        img_id = row['id']
        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(data_dir, img_info['file_name'])


        # Skip if the image file doesn't exist
        if not os.path.exists(img_path):
            print(f"Image file not found: {img_path}")
            continue

        # Load image
        image = io.imread(img_path)

        # Generate mask with priority-based label assignment
        annIds = coco.getAnnIds(imgIds=img_id, catIds=coco.getCatIds(), iscrowd=None)
        anns = coco.loadAnns(annIds)
        mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

        # Assign class values based on priority to prevent overlap issues
        for ann in sorted(anns, key=lambda x: label_priority.get(x['category_id'], -1)):
            class_id = ann['category_id']
            ann_mask = coco.annToMask(ann)
            mask[ann_mask == 1] = class_id  # Assign class id to the mask based on priority

        # Preprocess image and mask
        preprocessed_image = preprocess_image(image)
        preprocessed_mask = preprocess_mask(mask)

        # Apply augmentations if specified
        if augment:
            augmented_image, augmented_mask = apply_augmentations(preprocessed_image, preprocessed_mask)
        else:
            augmented_image, augmented_mask = preprocessed_image, preprocessed_mask

        # Define output path for the image and mask
        base_filename = os.path.splitext(img_info['file_name'])[0]
        img_output_path = os.path.join(image_output_dir, f"{base_filename}_processed.jpg")
        mask_output_path = os.path.join(mask_output_dir, f"{base_filename}_mask_colored.png") if save_masks and mask_output_dir else None


        # Ensure directories exist before saving
        os.makedirs(os.path.dirname(img_output_path), exist_ok=True)
        if mask_output_path:
            os.makedirs(os.path.dirname(mask_output_path), exist_ok=True)

        # Save the preprocessed image
        io.imsave(img_output_path, img_as_ubyte(augmented_image))

        # Save mask with colormap if specified
        if mask_output_path:
            plt.imsave(mask_output_path, augmented_mask, cmap=colormap)
            print(f"Saved mask with colormap for image ID {img_id} at {mask_output_path}")

print("Dataset processing function with colormap support is ready.")

Dataset processing function with colormap support is ready.


In [ ]:
process_and_save_dataset_multiclass(coco, train_metadata, train_image_dir, train_mask_dir, augment=True, save_masks=True, colormap='viridis')
print("Training dataset processed and saved with colormap.")


Saved mask with colormap for image ID 7 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks/images/t001/OiaMESWPyh_mask_colored.png
Saved mask with colormap for image ID 10 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks/images/t001/ZKnMahXQgJ_mask_colored.png
Saved mask with colormap for image ID 22 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks/images/t001/zgOfklUqJC_mask_colored.png
Saved mask with colormap for image ID 24 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks/images/t002/AyPUCFEHdl_mask_colored.png
Saved mask with colormap for image ID 28 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks/images/t002/SuPxFCvAob_mask_colored.png
Saved mask with colormap for image ID 36 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks/images/t002/pIHJimnzHb_mask_colored.png
Saved mask with colormap for image ID 51 at /content/drive/MyDrive/CV_9517/Da

In [ ]:
process_and_save_dataset_multiclass(coco, val_metadata, val_image_dir, val_mask_dir, augment=False, save_masks=True, colormap='viridis')
print("Validation dataset processed and saved with colormap.")

Saved mask with colormap for image ID 461 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/validation/masks/images/t023/ReJwfWAkuE_mask_colored.png
Saved mask with colormap for image ID 492 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/validation/masks/images/t023/shXzdbiwef_mask_colored.png
Saved mask with colormap for image ID 494 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/validation/masks/images/t023/wzUusCgQsh_mask_colored.png
Saved mask with colormap for image ID 521 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/validation/masks/images/t025/WhBgbmImGm_mask_colored.png
Saved mask with colormap for image ID 526 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/validation/masks/images/t025/gSGJQQyduo_mask_colored.png
Saved mask with colormap for image ID 539 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/validation/masks/images/t025/vsomsWqtXo_mask_colored.png
Saved mask with colormap for image ID 658 at /content/driv

In [ ]:
process_and_save_dataset_multiclass(coco, test_metadata, test_image_dir, test_mask_dir, augment=False, save_masks=True, colormap='viridis')
print("Test dataset processed and saved with colormap.")

Saved mask with colormap for image ID 122 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/testing/masks/images/t007/BBuJpHMVVi_mask_colored.png
Saved mask with colormap for image ID 130 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/testing/masks/images/t007/IyfuzGqJsY_mask_colored.png
Saved mask with colormap for image ID 142 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/testing/masks/images/t007/WuOEUwImuG_mask_colored.png
Saved mask with colormap for image ID 150 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/testing/masks/images/t007/bFoOkYgDwN_mask_colored.png
Saved mask with colormap for image ID 170 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/testing/masks/images/t007/woyemqnJnP_mask_colored.png
Saved mask with colormap for image ID 171 at /content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/testing/masks/images/t007/zhfCTvDAyO_mask_colored.png
Saved mask with colormap for image ID 180 at /content/drive/MyDrive/CV_9517/

In [ ]:
# Define function to get file paths
def get_file_paths(image_dir, mask_dir=None):

    image_paths = sorted(glob.glob(os.path.join(image_dir, '**', '*.jpg'), recursive=True))
    mask_paths = sorted(glob.glob(os.path.join(mask_dir, '**', '*.png'), recursive=True)) if mask_dir else None

    return image_paths, mask_paths

# Define directories (replace these with your actual paths if necessary)
train_image_dir = '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/images'
train_mask_dir = '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks'
val_image_dir = '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/validation/images'
val_mask_dir = '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/validation/masks'
test_image_dir = '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/testing/images'
test_mask_dir = '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/testing/masks'



# Fetch file paths
train_image_paths, train_mask_paths = get_file_paths(train_image_dir, train_mask_dir)
val_image_paths, val_mask_paths = get_file_paths(val_image_dir, val_mask_dir)
test_image_paths, test_mask_paths = get_file_paths(test_image_dir, test_mask_dir)  # Use mask_dir if test masks are available

# Sanity check: Print first few paths
print("Train Images:", train_image_paths[:5])
print("Train Masks:", train_mask_paths[:5])
print("Validation Images:", val_image_paths[:5])
print("Validation Masks:", val_mask_paths[:5])
print("Test Images:", test_image_paths[:5])
print("Test Masks:", test_mask_paths[:5])

Train Images: ['/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/images/images/t001/OiaMESWPyh_processed.jpg', '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/images/images/t001/ZKnMahXQgJ_processed.jpg', '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/images/images/t001/zgOfklUqJC_processed.jpg', '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/images/images/t002/AyPUCFEHdl_processed.jpg', '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/images/images/t002/SuPxFCvAob_processed.jpg']
Train Masks: ['/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks/images/t001/OiaMESWPyh_mask_colored.png', '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks/images/t001/ZKnMahXQgJ_mask_colored.png', '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks/images/t001/zgOfklUqJC_mask_colored.png', '/content/drive/MyDrive/CV_9517/Dataset/preprocess_2000/training/masks/images

In [ ]:
# Function to remap mask values
def remap_mask_values(mask):
    mapping = {30: 0, 90: 1, 136: 2, 215: 3}  # Replace with your actual mappings
    remapped_mask = np.copy(mask)
    for k, v in mapping.items():
        remapped_mask[mask == k] = v
    return remapped_mask

# Define function to process images and masks
def process_image_with_mask(image_path, mask_path):
    target_size = [224, 224]

    # Load and preprocess image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float32) / 255.0

    # Load and preprocess mask
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, target_size, method="nearest")
    mask = tf.cast(mask, tf.uint8)

    # Convert the mask to a NumPy array, apply remapping, and convert back to TensorFlow tensor
    mask = mask.numpy()  # Convert TensorFlow tensor to NumPy array
    mask = remap_mask_values(mask)  # Remap the mask values
    mask = tf.convert_to_tensor(mask)  # Convert back to TensorFlow tensor

    # Clip mask values to the valid range [0, 3] if necessary
    mask = tf.clip_by_value(mask, 0, 3)

    return image, mask

def create_dataset(image_paths, mask_paths, batch_size=32, shuffle=True):
    def _process_image_with_mask(img_path, mask_path):
        image, mask = tf.py_function(
            func=process_image_with_mask,
            inp=[img_path, mask_path],
            Tout=(tf.float32, tf.uint8)
        )
        image.set_shape([224, 224, 3])  # Set the shape explicitly
        mask.set_shape([224, 224, 1])   # Set the shape explicitly
        return image, mask

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    dataset = dataset.map(_process_image_with_mask, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Create TensorFlow datasets for train, validation, and test splits
train_dataset = create_dataset(train_image_paths, train_mask_paths, batch_size=32, shuffle=True)
val_dataset = create_dataset(val_image_paths, val_mask_paths, batch_size=32, shuffle=False)
test_dataset = create_dataset(test_image_paths, test_mask_paths, batch_size=32, shuffle=False)

# Sanity check: Ensure all masks have values in the expected range
for images, masks in train_dataset.take(1):
    print("Unique values in a sample mask:", np.unique(masks.numpy()))

# Sanity check: Display a batch from each dataset
for images, masks in train_dataset.take(1):
    print("Train batch - Image shape:", images.shape, ", Mask shape:", masks.shape)

for images, masks in val_dataset.take(1):
    print("Validation batch - Image shape:", images.shape, ", Mask shape:", masks.shape)

for images, masks in test_dataset.take(1):
    print("Test batch - Image shape:", images.shape, ", Mask shape:", masks.shape)

Unique values in a sample mask: [0 1 2 3]
Train batch - Image shape: (32, 224, 224, 3) , Mask shape: (32, 224, 224, 1)
Validation batch - Image shape: (32, 224, 224, 3) , Mask shape: (32, 224, 224, 1)
Test batch - Image shape: (32, 224, 224, 3) , Mask shape: (32, 224, 224, 1)
